In [1]:
#Setting up the data collection : import and API connexion variables:

import pandas as pd
import numpy as np
import requests
import json
import os
import bcrypt
import datetime
import pickle
from getpass import getpass
import time
import json

password = os.getenv('SORAREpwd')
api_key = os.getenv('SORARE_APIKEY')
salt = b"$2a$11$KnNxrgfQuLaxCfahjb3z3." 
encoded_password = password.encode('utf-8')
hashed_password = bcrypt.hashpw(encoded_password, salt)
hashed_password_string = hashed_password.decode()

In [2]:
email = getpass("login: ")

login:  ········


In [3]:
#Sign-in to API (1st Step)

sign_in_initial_url = "https://api.sorare.com/graphql"

# Define the headers for the initial sign-in request
headers = {
    "Content-Type": "application/json"
}

# Construct the payload for the initial sign-in mutation
sign_in_initial_payload = {
    "operationName": "SignInMutation",
    "variables": {
        "input": {
            "email": email,
            "password": hashed_password_string
        }
    },
    "query": """
        mutation SignInMutation($input: signInInput!) {
            signIn(input: $input) {
                otpSessionChallenge
                errors {
                    message
                }
            }
        }
    """
}

# Make the initial sign-in request
sign_in_initial_response = requests.post(sign_in_initial_url, headers=headers, data=json.dumps(sign_in_initial_payload))
sign_in_initial_json_response = sign_in_initial_response.json()

# Handle the initial sign-in response
otp_session_challenge = sign_in_initial_json_response["data"]["signIn"]["otpSessionChallenge"]

In [4]:
otp_2FA = getpass('2FA:')

2FA: ········


In [5]:
# Sign-in with 2FA request
sign_in_2fa_url = "https://api.sorare.com/federation/graphql"

# Construct the payload for the sign-in with 2FA mutation
sign_in_2fa_payload = {
    "operationName": "SignInMutation",
    "variables": {
        "input": {
            "otpSessionChallenge": otp_session_challenge,
            "otpAttempt": otp_2FA
        }
    },
    "query": """
        mutation SignInMutation($input: signInInput!) {
            signIn(input: $input) {
                currentUser {
                    slug
                    jwtToken(aud: "<YourAud>") {
                        token
                        expiredAt
                    }
                }
                errors {
                    message
                }
            }
        }
    """
}

# Make the sign-in with 2FA request
sign_in_2fa_response = requests.post(sign_in_2fa_url, headers=headers, data=json.dumps(sign_in_2fa_payload))
sign_in_2fa_json_response = sign_in_2fa_response.json()

In [6]:
user_key = sign_in_2fa_json_response['data']['signIn']['currentUser']['jwtToken']['token']

In [7]:
#Function for Querying Sorare API from Python. Input is the GraphQL Query to put in it

def graphql_request(query):
    # Replace with your API URL
    url = "https://api.sorare.com/federation/graphql"

    # Define the headers for the request
    headers = {
        "Content-Type": "application/json",
        "APIKEY": api_key
    }

    # Create the payload with the query
    payload = {
        "query": query
    }

    # Make the GraphQL request
    response = requests.post(url, headers=headers, json=payload)

    # Parse the JSON response
    json_data = response.json()

    # Return the JSON response
    return json_data

In [8]:
# Query to fetch all cards from Clubs (target is to loop that query on a list of all clubs from a League)

cardsbyplayer_query = '''
    query getAllLimitedCardsByplayers {
  allCards(
    rarities: [limited]
    playerSlugs: ["memphis-depay"]) {
    nodes {
      id
      assetId
      name
      slug
      rarity
      createdAt
      seasonYear
      grade
      power
      xp
      serialNumber
      shirtNumber
      customCardEditionName
      player {
        slug
        activeClub {
          name
        }
      }
      ownershipHistory {
        id
        token {
          id
          ethereumId
        }
        from
        transferType
        priceWei
        priceFiat {
          eur
        }
        
        user {
          id
          slug
        }
      }
    }
    pageInfo {
      endCursor
    }
  }
}
  '''

In [9]:
test_2 = graphql_request(cardsbyplayer_query)
test_2

{'data': {'allCards': {'nodes': [{'id': 'Card:00b2e6c3-7984-4044-9b0b-9daed89a4216',
     'assetId': '0x0400ab4cc03a81cb0993e010782e10338470b6f1babc46621223bff33bff4c49',
     'name': 'Memphis Depay 2022-23 • Limited 232/1000',
     'slug': 'memphis-depay-2022-limited-232',
     'rarity': 'limited',
     'createdAt': '2023-03-14T20:29:24Z',
     'seasonYear': 2022,
     'grade': 11,
     'power': '1.075',
     'xp': 7435,
     'serialNumber': 232,
     'shirtNumber': 9,
     'customCardEditionName': None,
     'player': {'slug': 'memphis-depay',
      'activeClub': {'name': 'Club Atlético de Madrid'}},
     'ownershipHistory': [{'id': 'Owner:ea678cd7-e3da-43f3-a4f6-a0a382aec800',
       'token': {'id': 'Starkware::MintableERC721Token:b03ea449-6393-4818-88aa-94bb0816953b',
        'ethereumId': '41239674665246472915523956160885740841863443921726172926842984833543612172672'},
       'from': '2023-04-18T18:22:04Z',
       'transferType': 'REWARD',
       'priceWei': '0',
       'priceFiat

In [10]:
test['data']['allCards']['pageInfo']['endCursor']

NameError: name 'test' is not defined

In [11]:
# Functions For Looping Queries: 

def build_query_first50(player):
    query = '''
    query getAllLimitedCardsByPlayers {
  allCards(
    rarities: [limited]
    playerSlugs: "%s"
  ) {
    nodes {
      id
      assetId
      name
      slug
      rarity
      createdAt
      seasonYear
      grade
      power
      xp
      serialNumber
      shirtNumber
      customCardEditionName
      player {
        slug
        activeClub {
          name
        }
      }
      ownershipHistory {
        from
        transferType
        priceWei
        priceFiat {
          eur
        }
        user {
          id
          slug
        }
      }
    }
    pageInfo {
      endCursor
    }
  }
}
  ''' % player
    
    return query
    
def build_query_next50(cursor,player):
    query = '''
    query getAllLimitedCardsByPlayers {
  allCards(
    rarities: [limited]
    playerSlugs: "%s"
    after: "%s"
  ) {
    nodes {
      id
      assetId
      name
      slug
      rarity
      createdAt
      seasonYear
      grade
      power
      xp
      serialNumber
      shirtNumber
      customCardEditionName
      player {
        slug
        activeClub {
          name
        }
      }
      ownershipHistory {
        from
        transferType
        priceWei
        priceFiat {
          eur
        }
        user {
          id
          slug
        }
      }
    }
    pageInfo {
      endCursor
    }
  }
}
  ''' % (player,cursor)
    return query


In [12]:
transactions_df = pd.read_csv('all_cards_transactions_la_liga.csv')

In [53]:
transactions_df.dropna(subset=['player_slug'], inplace=True)
la_liga_players_list = transactions_df['player_slug'].unique().tolist()

In [55]:
len(la_liga_players_list)

909

In [21]:
# Function for looping in order to get all cards (API Paginated per 50 results)

def get_all_cards(player):
    responses = []
    query = build_query_first50(player)
    response = graphql_request(query)
    responses.append(response)
    end_cursor = response['data']['allCards']['pageInfo']['endCursor']
    start_cursor = end_cursor
    x=0
    while start_cursor:
        query = build_query_next50(start_cursor,player)
        response = graphql_request(query)
        responses.append(response)
        end_cursor = response['data']['allCards']['pageInfo']['endCursor']
        start_cursor = end_cursor
        if x%10 == 0: 
            print('x =',x, end_cursor, datetime.datetime.now())
            with open('second_card_records_backup.pickle', 'wb') as handle:
                pickle.dump(responses, handle, protocol=pickle.HIGHEST_PROTOCOL)
        x+=1
        if x==100:
            break
    return responses



In [ ]:

test_list_of_players = ["matheus-santos-carneiro-da-cunha",
                 "jude-bellingham"
                ]

In [56]:
'''

all_cards_per_players = []
for player in la_liga_players_list:
    all_cards_per_players.append(get_all_cards(player))
    with open('player_card_records_backup.pickle', 'wb') as handle:
                pickle.dump(all_cards_per_players, handle, protocol=pickle.HIGHEST_PROTOCOL)'''


x = 0 WyIxNTAyMzI2My0wZTU2LTQyOTktYmEzZS04NjAxYWYwOTRjYTYiXQ 2024-02-11 14:06:35.910142
x = 10 WyI3Nzk3NDM1Ny1mZWE0LTQ2YjQtYjkwOS1lNTY3MDhjZTBiZjIiXQ 2024-02-11 14:06:38.751292
x = 20 WyJkYWFkODgzOC05NGY3LTRlNDItYmZkYi0zYTQ2ZDE0ZmMzNjIiXQ 2024-02-11 14:06:41.900490
x = 0 WyIxYzg5MTZkMC05NjRmLTQ4NmUtYmZkYy04MTFiNjMzNmM1NjkiXQ 2024-02-11 14:06:43.740474
x = 10 WyJiMzkxYzYzMy00MjRmLTQ4NjctYjIwMi0yNGExZmNjODNlYTEiXQ 2024-02-11 14:06:46.443913
x = 0 WyIxM2FjZTMzNi1jYzgyLTRiN2UtYTQzMC02MDU2MjViYjRhMDQiXQ 2024-02-11 14:06:48.771694
x = 10 WyI3NmQzMzJiMi1jZjdjLTQ4OGItYmE4Yy04OTI1OGRiYWVmMGEiXQ 2024-02-11 14:06:51.659922
x = 20 WyJlMmY4ZWIxMi1kZDJjLTRlZjMtOWFlOC1jMGI1ZDk3Nzg3MWUiXQ 2024-02-11 14:06:54.581934
x = 0 WyIxYjkwYTJmNy1hOGMzLTRmYzgtOGY3Yy0xNTQ0M2RhOGZiYmUiXQ 2024-02-11 14:06:56.229627
x = 10 WyI5MGZhOTU1Ni1mN2JkLTRlMjYtYmFjYS01ZGE0MDU5NmU5OGUiXQ 2024-02-11 14:06:59.409270
x = 20 WyJmZjQ2ZjBmZS1lMDZhLTRmOTgtOTJmZi00YWEzMjA2MGY1OGQiXQ 2024-02-11 14:07:02.383879
x = 0 WyIxN2JlYmU3MC0xNWN

In [59]:
len(all_cards_per_players)

909

In [58]:
with open('player_card_records.pickle', 'wb') as file:
    pickle.dump(all_cards_per_players, file)

In [61]:
total_nodes = 0

for card_list in all_cards_per_players:
    for card in card_list:
        nodes = card['data']['allCards']['nodes']
        total_nodes += len(nodes)

print(total_nodes)

617654


In [63]:
player_card_data = []

for i, card_list in enumerate(all_cards_per_players):
    player_name = la_liga_players_list[i]
    
    for card_dict in card_list:
        nodes = card_dict['data']['allCards']['nodes']
        
        for card in nodes:
            for history in card.get('ownershipHistory', []):
                if history and history.get('user') is not None:  # Check if history and history['user'] are not None
                    card_info = {
                        'team_name': None,  # Add a default value for 'team_slug',
                        'card_id': card['id'],
                        'asset_id': card['assetId'],
                        'card_name': card['name'],
                        'season_year': card['seasonYear'],
                        'card_slug': card['slug'],
                        'rarity': card['rarity'],
                        'card_creation_date': card['createdAt'],
                        'grade': card['grade'],
                        'power': card['power'],
                        'xp': card['xp'],
                        'serial_number': card['serialNumber'],
                        'shirt_number': card['shirtNumber'],
                        'special_edition': card['customCardEditionName'],
                        'transaction_date': history['from'],
                        'transaction_type': history['transferType'],
                        'price_wei': history['priceWei'],
                        'price_eur': history['priceFiat']['eur'],
                        'user_id': history['user']['id'],
                        'user_slug': history['user']['slug'],
                        'player_slug': player_name
                    }
                    player_info = card.get('player')
                    if player_info and player_info.get('activeClub') and player_info.get('slug'):
                        card_info['team_name'] = player_info['activeClub']['name']
                    player_card_data.append(card_info)

all_transactions_df = pd.DataFrame(player_card_data)


In [64]:
all_transactions_df.shape

(1679833, 21)

In [66]:
all_transactions_df.head()

,team_name,card_id,asset_id,card_name,season_year,card_slug,rarity,card_creation_date,grade,power,...,serial_number,shirt_number,special_edition,transaction_date,transaction_type,price_wei,price_eur,user_id,user_slug,player_slug
0,Real Madrid CF,Card:00035848-b60d-4999-832c-e4dd8e939007,0x040004752c83837b25107500c71efa485aafb6cf1b82...,Nacho 2021-22 • Limited 218/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-218,limited,2021-11-25T03:40:39Z,7,1.045,...,218,6,None,2022-01-18T10:23:04Z,REFERRAL,0,0.000000,User:dff07274-2664-4ed9-9308-4de48a2b8bec,serveg,jose-ignacio-fernandez-iglesias
1,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31T08:58:22Z,13,1.075,...,488,6,None,2022-07-19T09:23:20Z,ENGLISH_AUCTION,6500000000000000,9.747140,User:ee426172-1955-4317-98d7-480476d0bb28,anonymous-8f8df118-a1d8-493d-9a5e-98997ed6450b,jose-ignacio-fernandez-iglesias
2,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31T08:58:22Z,13,1.075,...,488,6,None,2022-07-19T09:36:47Z,SINGLE_SALE_OFFER,5100000000000000,7.647756,User:f831ecc8-990a-42e7-94b5-6656536685f5,bugggyman-ua,jose-ignacio-fernandez-iglesias
3,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31T08:58:22Z,13,1.075,...,488,6,None,2022-07-19T11:55:20Z,SINGLE_SALE_OFFER,5400000000000000,8.116470,User:9d7ae8cb-c48e-4e5b-8715-eae0826c06c1,junmisugi84-e6f8dfdd-08b3-463f-a2b5-d5870eb74deb,jose-ignacio-fernandez-iglesias
4,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31T08:58:22Z,13,1.075,...,488,6,None,2023-03-14T10:28:55Z,SINGLE_BUY_OFFER,2800000000000000,4.356016,User:2d9f44fb-d504-4159-96bd-89deaa61e585,alvaropm019,jose-ignacio-fernandez-iglesias


In [72]:
#all_transactions_df.to_csv('all_players_cards_transactions_la_liga.csv', index=False)

In [73]:
jude_cards = all_transactions_df[all_transactions_df['player_slug']=='jude-bellingham']

In [74]:
jude_cards['season_year'].value_counts()

season_year
2021    1564
2022    1061
2023     421
Name: count, dtype: int64

In [55]:
#test query for fetching a player clubs over the thim

player_all_clubs = '''
query PlayerAllClubs {
  football {
    player(slug: "memphis-depay") {
      slug
      id
      clubMemberships{
        club{
          slug
          id
        }
        startDate
        endDate
      }
    }
  }
}
'''

In [56]:
test_club = graphql_request(player_all_clubs)
test_club

{'data': {'football': {'player': {'slug': 'memphis-depay',
    'id': 'Player:95e34a9f-d308-4abf-b432-9bf89f68577c',
    'clubMemberships': [{'club': {'slug': 'manchester-united-manchester',
       'id': 'Club:8d4a17c5-b1c8-4436-8f7e-b99fab1b1c99'},
      'startDate': '2015-07-01T00:00:00+00:00',
      'endDate': '2017-01-20T00:00:00+00:00'},
     {'club': {'slug': 'psv-eindhoven',
       'id': 'Club:0501ee25-5d5f-445b-9c4a-f2912cde4328'},
      'startDate': '2011-09-01T00:00:00+00:00',
      'endDate': '2015-07-01T00:00:00+00:00'},
     {'club': {'slug': 'olympique-lyonnais-lyon',
       'id': 'Club:42b054fb-1ba3-4547-857b-97b3f229abc7'},
      'startDate': '2017-01-20T00:00:00+00:00',
      'endDate': '2021-06-30T00:00:00+00:00'},
     {'club': {'slug': 'atletico-madrid-madrid',
       'id': 'Club:f92833c9-4e18-413e-8a3c-91c06f77abb3'},
      'startDate': '2023-01-20T00:00:00+00:00',
      'endDate': None},
     {'club': {'slug': 'barcelona-barcelona',
       'id': 'Club:f115e8de-5e7a

In [9]:
def get_clubs_history(player):
    query = '''
    query PlayerAllClubs {{
      football {{
        player(slug: "{player}") {{
          slug
          id
          position
          age
          u23Eligible
          u23EligibleUntil
          clubMemberships {{
            club {{
              slug
              id
            }}
            startDate
            endDate
          }}
        }}
      }}
    }}
    '''.format(player=player)
    return query
 

In [10]:
def get_all_clubs(list_of_players):
    clubs_data = []  # Adapted to a list
    
    for index, player in enumerate(list_of_players):
        try:
            time.sleep(0.1)
            print("new_player")
            query = get_clubs_history(player)
            response = graphql_request(query)
            clubs_data.append(response)  # Append to the list instead of dictionary

            if response is None:
                print(f"Error fetching data for player {index}: Response is None")
                continue

        except KeyError as e:
            print(f"Error fetching data for player {index}: Missing key {str(e)}")
            break

    return clubs_data


In [11]:
list_of_players_test = ['paulo-dino-gazzaniga','toni-fuidias-ribera','aleix-garcia-serrano']

In [16]:
test_pos = get_all_clubs(list_of_players_test)

new_player
new_player
new_player


In [17]:
test_pos

[{'data': {'football': {'player': {'slug': 'paulo-dino-gazzaniga',
     'id': 'Player:dc8f1b84-bee3-4abf-a228-09c0981f34af',
     'position': 'Goalkeeper',
     'age': 32,
     'u23Eligible': False,
     'u23EligibleUntil': None,
     'clubMemberships': [{'club': {'slug': 'southampton-southampton-hampshire',
        'id': 'Club:dc5e0a7c-ac33-4929-907e-601aa3019b1c'},
       'startDate': '2012-07-20T00:00:00+00:00',
       'endDate': '2016-07-29T00:00:00+00:00'},
      {'club': {'slug': 'southampton-southampton-hampshire',
        'id': 'Club:dc5e0a7c-ac33-4929-907e-601aa3019b1c'},
       'startDate': '2017-07-01T00:00:00+00:00',
       'endDate': '2017-08-23T00:00:00+00:00'},
      {'club': {'slug': 'gillingham-gillingham-kent',
        'id': 'Club:fa2f4470-eb27-4023-b424-60741ec2a3ae'},
       'startDate': '2011-07-27T00:00:00+00:00',
       'endDate': '2012-07-19T00:00:00+00:00'},
      {'club': {'slug': 'rayo-vallecano-madrid',
        'id': 'Club:e30c8b23-e744-4fb4-b135-ee1204c41fe

In [12]:
all_transactions_df = pd.read_csv('all_players_cards_transactions_la_liga.csv', dtype={20: str})

In [13]:
all_transactions_df.head()

,team_name,card_id,asset_id,card_name,season_year,card_slug,rarity,card_creation_date,grade,power,...,serial_number,shirt_number,special_edition,transaction_date,transaction_type,price_wei,price_eur,user_id,user_slug,player_slug
0,Real Madrid CF,Card:00035848-b60d-4999-832c-e4dd8e939007,0x040004752c83837b25107500c71efa485aafb6cf1b82...,Nacho 2021-22 • Limited 218/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-218,limited,2021-11-25T03:40:39Z,7,1.045,...,218,6,NaN,2022-01-18T10:23:04Z,REFERRAL,0,0.000000,User:dff07274-2664-4ed9-9308-4de48a2b8bec,serveg,jose-ignacio-fernandez-iglesias
1,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31T08:58:22Z,13,1.075,...,488,6,NaN,2022-07-19T09:23:20Z,ENGLISH_AUCTION,6500000000000000,9.747140,User:ee426172-1955-4317-98d7-480476d0bb28,anonymous-8f8df118-a1d8-493d-9a5e-98997ed6450b,jose-ignacio-fernandez-iglesias
2,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31T08:58:22Z,13,1.075,...,488,6,NaN,2022-07-19T09:36:47Z,SINGLE_SALE_OFFER,5100000000000000,7.647756,User:f831ecc8-990a-42e7-94b5-6656536685f5,bugggyman-ua,jose-ignacio-fernandez-iglesias
3,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31T08:58:22Z,13,1.075,...,488,6,NaN,2022-07-19T11:55:20Z,SINGLE_SALE_OFFER,5400000000000000,8.116470,User:9d7ae8cb-c48e-4e5b-8715-eae0826c06c1,junmisugi84-e6f8dfdd-08b3-463f-a2b5-d5870eb74deb,jose-ignacio-fernandez-iglesias
4,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31T08:58:22Z,13,1.075,...,488,6,NaN,2023-03-14T10:28:55Z,SINGLE_BUY_OFFER,2800000000000000,4.356016,User:2d9f44fb-d504-4159-96bd-89deaa61e585,alvaropm019,jose-ignacio-fernandez-iglesias


In [14]:
all_transactions_df.dropna(subset=['player_slug'], inplace=True)
la_liga_players_list = all_transactions_df['player_slug'].unique().tolist()

In [18]:
len(la_liga_players_list)

909

In [19]:
clubs_history = get_all_clubs(la_liga_players_list)

new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player
new_player

In [137]:
# Rest of your code that loops through the list and creates the DataFrame
player_slug_list = []
position_list = []
age_list = []
u23_eligible_list = []
U23_until_list = []
club_slug_list = []
start_date_list = []
end_date_list = []

# Loop through the list and extract information for each player
for item in clubs_history:
    if isinstance(item, dict):  # Check if item is a dictionary
        player_data = item['data']['football']['player']
        slug = player_data['slug']
        position = player_data['position']
        age = player_data['age']
        u23_eligible = player_data['u23Eligible']
        u23_until = player_data['u23EligibleUntil']
        club_memberships = player_data['clubMemberships']

        # Loop through the club memberships and extract information
        for membership in club_memberships:
            player_slug_list.append(slug)
            position_list.append(position)
            age_list.append(age)
            u23_eligible_list.append(u23_eligible)
            u23_until_list.append(u23_until)
            club_slug_list.append(membership['club']['slug'])
            start_date_list.append(membership['startDate'])
            end_date_list.append(membership['endDate'])

# Create a DataFrame from the lists
clubs_hist_df = pd.DataFrame({
    'player_slug': player_slug_list,
    'position': position_list,
    'age': ade_list,
    'u23_eligible': u23_eligible_list,
    'u23_until': u23_until_list,
    'club_slug': club_slug_list,
    'contract_start_date': start_date_list,
    'contract_end_date': end_date_list
})


In [138]:
clubs_hist_df.shape

(6371, 4)

In [139]:
print(clubs_hist_df.dtypes)

playerslug             object
club_slug              object
contract_start_date    object
contract_end_date      object
dtype: object


In [142]:
# Assuming the DataFrame is called clubs_hist_df

# Convert the "contract_start_date" and "contract_end_date" columns to datetime format
clubs_hist_df['contract_start_date'] = pd.to_datetime(clubs_hist_df['contract_start_date'])
clubs_hist_df['contract_end_date'] = pd.to_datetime(clubs_hist_df['contract_end_date'], format='%Y-%m-%dT%H:%M:%S%z')

# Fill empty values in "contract_end_date" column with the date "2024-06-30 00:00:00+00:00"
clubs_hist_df['contract_end_date'].fillna(pd.to_datetime('2024-06-30 00:00:00+00:00'), inplace=True)


In [144]:
clubs_hist_df.head(50)

,playerslug,club_slug,contract_start_date,contract_end_date
0,jose-ignacio-fernandez-iglesias,real-madrid-madrid,2011-04-23 00:00:00+00:00,2024-06-30 00:00:00+00:00
1,jose-ignacio-fernandez-iglesias,real-madrid-ii-madrid,2010-07-01 00:00:00+00:00,2013-06-30 00:00:00+00:00
2,luis-federico-lopez-andugar,real-madrid-u19-madrid,2018-07-01 00:00:00+00:00,2021-06-30 00:00:00+00:00
3,luis-federico-lopez-andugar,real-madrid-madrid,2020-07-18 00:00:00+00:00,2023-07-25 00:00:00+00:00
4,luis-federico-lopez-andugar,real-madrid-ii-madrid,2019-08-01 00:00:00+00:00,2023-07-25 00:00:00+00:00
5,luis-federico-lopez-andugar,real-madrid-madrid,2024-07-01 00:00:00+00:00,2024-06-30 00:00:00+00:00
6,luis-federico-lopez-andugar,mirandes-miranda-de-ebro,2023-07-25 00:00:00+00:00,2024-06-30 00:00:00+00:00
7,lucas-vazquez-iglesias,espanyol-barcelona,2015-07-01 00:00:00+00:00,2015-07-02 00:00:00+00:00
8,lucas-vazquez-iglesias,espanyol-barcelona,2014-08-20 00:00:00+00:00,2015-07-01 00:00:00+00:00
9,lucas-vazquez-iglesias,real-madrid-madrid,2014-03-01 00:00:00+00:00,2014-08-20 00:00:00+00:00


In [146]:
clubs_hist_df.to_csv('clubs_hist.csv', index=False)

#### Fetching the gameweeks starts and ends time stamps 

In [21]:
#unit query for GW:  

gw_query = '''
    query nodes{
  football{
    so5{
        so5Fixtures(after:"WyIyMDIzLTAzLTA3IDE1OjAwOjAwLjAwMDAwMDAwMCBVVEMiLCJiMTExNDc4Zi1jNDFjLTQ5MDItYjc4ZS1lNzc5MDgwOWE1YTQiXQ"){
          nodes{
          gameWeek
          startDate
          endDate
        }
        pageInfo{
          endCursor
        }
          }
            }
          } 
}
  '''


In [23]:
#test: 
gws = graphql_request(gw_query)

In [25]:
display(gws)

{'data': {'football': {'so5': {'so5Fixtures': {'nodes': [{'gameWeek': 351,
       'startDate': '2023-03-03T15:00:00Z',
       'endDate': '2023-03-07T14:59:59Z'},
      {'gameWeek': 350,
       'startDate': '2023-02-28T15:00:00Z',
       'endDate': '2023-03-03T14:59:59Z'},
      {'gameWeek': 349,
       'startDate': '2023-02-24T15:00:00Z',
       'endDate': '2023-02-28T14:59:59Z'},
      {'gameWeek': 348,
       'startDate': '2023-02-21T15:00:00Z',
       'endDate': '2023-02-24T14:59:59Z'},
      {'gameWeek': 347,
       'startDate': '2023-02-17T15:00:00Z',
       'endDate': '2023-02-21T14:59:59Z'},
      {'gameWeek': 346,
       'startDate': '2023-02-14T15:00:00Z',
       'endDate': '2023-02-17T14:59:59Z'},
      {'gameWeek': 345,
       'startDate': '2023-02-10T15:00:00Z',
       'endDate': '2023-02-14T14:59:59Z'},
      {'gameWeek': 344,
       'startDate': '2023-02-07T15:00:00Z',
       'endDate': '2023-02-10T14:59:59Z'},
      {'gameWeek': 343,
       'startDate': '2023-02-03T15:00

In [26]:
gws['data']['football']['so5']['so5Fixtures']['pageInfo']['endCursor']

'WyIyMDIyLTA5LTAyIDEwOjAwOjAwLjAwMDAwMDAwMCBVVEMiLCJjNTc0YmQ4Ny04NzA4LTRhY2QtODhmZS1hNTZiMDIwZTc3OTciXQ'

In [29]:
# loop queries: 

def build_query_first50_GW():
    query = '''
    query nodes{
  football{
    so5{
        so5Fixtures{
          nodes{
          gameWeek
          startDate
          endDate
        }
        pageInfo{
          endCursor
        }
          }
            }
          } 
}
  '''
    return query
    
def build_query_next50_GW(cursor):
    query = '''
query nodes{
  football{
    so5{
        so5Fixtures(after:"%s"){
          nodes{
          gameWeek
          startDate
          endDate
        }
        pageInfo{
          endCursor
        }
          }
            }
          } 
}
''' % cursor
    return query


In [34]:
def get_all_gws():
    responses = []
    query = build_query_first50_GW()
    response = graphql_request(query)
    responses.append(response)
    end_cursor = response['data']['football']['so5']['so5Fixtures']['pageInfo']['endCursor']
    start_cursor = end_cursor
    x=0
    while start_cursor:
        query = build_query_next50_GW(start_cursor)
        response = graphql_request(query)
        responses.append(response)
        end_cursor = response['data']['football']['so5']['so5Fixtures']['pageInfo']['endCursor']
        start_cursor = end_cursor
        if x%10 == 0: 
            print('x =',x, end_cursor, datetime.datetime.now())
        x+=1
        if x==100:
            break
    return responses

In [35]:
gameweeks = get_all_gws()

x = 0 WyIyMDIzLTAzLTA3IDE1OjAwOjAwLjAwMDAwMDAwMCBVVEMiLCJiMTExNDc4Zi1jNDFjLTQ5MDItYjc4ZS1lNzc5MDgwOWE1YTQiXQ 2024-02-14 19:01:07.936495


In [36]:
gameweeks

[{'data': {'football': {'so5': {'so5Fixtures': {'nodes': [{'gameWeek': 451,
        'startDate': '2024-02-16T15:00:00Z',
        'endDate': '2024-02-20T14:59:59Z'},
       {'gameWeek': 450,
        'startDate': '2024-02-13T15:00:00Z',
        'endDate': '2024-02-16T14:59:59Z'},
       {'gameWeek': 449,
        'startDate': '2024-02-09T15:00:00Z',
        'endDate': '2024-02-13T14:59:59Z'},
       {'gameWeek': 448,
        'startDate': '2024-02-06T15:00:00Z',
        'endDate': '2024-02-09T14:59:59Z'},
       {'gameWeek': 447,
        'startDate': '2024-02-02T15:00:00Z',
        'endDate': '2024-02-06T14:59:59Z'},
       {'gameWeek': 446,
        'startDate': '2024-01-30T15:00:00Z',
        'endDate': '2024-02-02T14:59:59Z'},
       {'gameWeek': 445,
        'startDate': '2024-01-26T15:00:00Z',
        'endDate': '2024-01-30T14:59:59Z'},
       {'gameWeek': 444,
        'startDate': '2024-01-23T15:00:00Z',
        'endDate': '2024-01-26T14:59:59Z'},
       {'gameWeek': 443,
        'sta

In [94]:
nodes_list = []
for i in range(len(gameweeks)):
    nodes = gameweeks[i]['data']['football']['so5']['so5Fixtures']['nodes']
    nodes_list.append(nodes)

gameweeks_df = pd.concat([pd.DataFrame(nodes) for nodes in nodes_list], ignore_index=True)

In [97]:
gameweeks_df.head()

,gameWeek,startDate,endDate
0,451,2024-02-16T15:00:00Z,2024-02-20T14:59:59Z
1,450,2024-02-13T15:00:00Z,2024-02-16T14:59:59Z
2,449,2024-02-09T15:00:00Z,2024-02-13T14:59:59Z
3,448,2024-02-06T15:00:00Z,2024-02-09T14:59:59Z
4,447,2024-02-02T15:00:00Z,2024-02-06T14:59:59Z


In [99]:
gameweeks_df.shape

(449, 3)

In [100]:
gameweeks_df.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
444    False
445    False
446    False
447    False
448    False
Length: 449, dtype: bool

In [105]:
gameweeks_df.head()

,gameWeek,startDate,endDate
0,451,2024-02-16T15:00:00Z,2024-02-20T14:59:59Z
1,450,2024-02-13T15:00:00Z,2024-02-16T14:59:59Z
2,449,2024-02-09T15:00:00Z,2024-02-13T14:59:59Z
3,448,2024-02-06T15:00:00Z,2024-02-09T14:59:59Z
4,447,2024-02-02T15:00:00Z,2024-02-06T14:59:59Z


In [91]:
len(gameweeks)

10

In [106]:
unique_gameweek_numbers = gameweeks_df['gameWeek'].unique()
print(unique_gameweek_numbers)

[451 450 449 448 447 446 445 444 443 442 441 440 439 438 437 436 435 434
 433 432 431 430 429 428 427 426 425 424 423 422 421 420 419 418 417 416
 415 414 413 412 411 410 409 408 407 406 405 404 403 402 401 400 399 398
 397 396 395 394 393 392 391 390 389 388 387 386 385 384 383 382 381 380
 379 378 377 376 375 374 373 372 371 370 369 368 367 366 365 364 363 362
 361 360 359 358 357 356 355 354 353 352 351 350 349 348 347 346 345 344
 343 342 341 340 339 338 337 336 335 334 333 332 331 330 329 328 327 326
 325 324 323 322 321 320 319 318 317 316 315 314 313 312 311 310 309 308
 307 306 305 304 303 302 301 300 299 298 297 296 295 294 293 292 291 290
 289 288 287 286 285 284 283 282 281 280 279 278 277 276 275 274 273 272
 271 270 269 268 267 266 265 264 263 262 261 260 259 258 257 256 255 254
 253 252 251 250 249 248 247 246 245 244 243 242 241 240 239 238 237 236
 235 234 233 232 231 230 229 228 227 226 225 224 223 222 221 220 219 218
 217 216 215 214 213 212 211 210 209 208 207 206 20

In [107]:
gameweeks_df = gameweeks_df.rename(columns={'gameWeek': 'gameweek_number', 'startDate': 'gameweek_start', 'endDate': 'gameweek_end'})

In [108]:
gameweeks_df.head()

,gameweek_number,gameweek_start,gameweek_end
0,451,2024-02-16T15:00:00Z,2024-02-20T14:59:59Z
1,450,2024-02-13T15:00:00Z,2024-02-16T14:59:59Z
2,449,2024-02-09T15:00:00Z,2024-02-13T14:59:59Z
3,448,2024-02-06T15:00:00Z,2024-02-09T14:59:59Z
4,447,2024-02-02T15:00:00Z,2024-02-06T14:59:59Z


In [109]:
gameweeks_df.to_csv('gameweeks_dates.csv', index=False)